# Understanding tf.Example w/ TF 2.0 

First, understand this is the world's WORST named class tf.Example - not an example - it is an Example of data (I guess).   TF Version - there are issues here!!!
- If you are using SageMaker, you may run into version issues;  you can unintall & reinstall, but MAY run into SageMaker compatibility issues - as of 20190930
- If you run w/ TF v 1.14, you can try part of this with Eager Execution, but you'll run into some problems with the data types
- Or, you can run most of this with TF v 1.14 with NO Eager - 

If you start getting errors - you need to understand if you are in Eager or Graph Mode and what version you're on. 


https://www.tensorflow.org/beta/tutorials/load_data/images  
https://www.tensorflow.org/tutorials/load_data/tf_records

You will need these skills!  What is an image, tf.Example, serialized example etc.  
You won't get far with served models without this understanding.

## TensorFlow 2.0 Beta
There is an issue with feature2 in the tutorial - a byte string.   It will definitely work if you get the right combination of versions and Eager Execution - but it's confusing and it's not really critical to your overall objective - so, I commented OUT including feature2 in features.   This github issue documents the problem:
https://github.com/tensorflow/tensorflow/issues/27181

### TensorFlow Versions
Restart the Kernel everytime you switch versions or Eager settings - if you just re-run, it won't take effect.
* v 1.14 - withOUT Eager Execution: There isn't much value in working w/ TF v1.x witout Eager on since Eager is the future of 2.0.  But, there are things to be learned.   You'll gain a better understanding of Eager.  This is particularly valuable if you have worked in Graph Mode w/ TF 1.x for years
  - f2.numpy() won't work;  f2 is a byte string - which has no .numpy() method
  - writer.write (writing the tfrecord) - no error but it doesn't do anything.   This probably needs to be executed in a graph.   This would be an old pattern (graph) so it's not worth dealing with.
* v 1.14 - WITH Eager Execution:
  - the graph step won't work of course
  - the f2.numpy() DOES work because f2 is no longer a byte string, it is now a Tensor dtype=string.  Being a Tensor, there IS a .numpy() method to return bytes
  - the TEST - tf_serialize_example(f0,f1,f2,f3) Won't work - you'll get the wierd can't copy bytes error.  This is probably because this py_function() runs in graph mode.  So this is probably to be expected -- it's not in the tutorial - I added it :)
  - ds.map(tf_serialize_example) - DOES work (probably because I took out the features = feature2 line
  - And, it DOES write a tfrecord file
* v 2.0 beta (20191001)
  - Everything work (with feature2 commented out in the features= function)
  - even the TEST - tf_serialize_example(f0,f1,f2,f3)
* v 2.0 (20191001) pip install tensororflow-gpu gives you 1.14, not 2.0, best you can get today is 2.0b1

#### Conclusion - as of 20191001, TensorFlow 2.0 is supposedly out - but it's not really.   Everything works with 1.14 & Eager + it's compatible with SageMaker - so stick with v 1.14 for today 

### Do this THIRD, first, do UnderstandingTF_IO & UnderstandingImages.ipynb

### tf.Example w/ SSD Images
There is another similar notebook:  UnderstandingObjectDetectionExamples  
Work through these basic concepts then you can apply to the SSD Images in this other notebook.   Also, this other notebook (along with Make_TFRecords) will show you how to make tfrecord files - which are based on tf.Example


### tf.train.Example
Can't find python API documentation - what are the methods?  
- FromString
- SerializeToString
- ? ParseSingle
- ? Parse

In [ ]:
import os, pathlib
import random
import IPython.display as display

import numpy as np
import matplotlib.pyplot as plt

# be sure to reset the kernel
# !pip uninstall -y tensorflow-gpu
# !pip install -q tensorflow==1.14
# !pip install -q tensorflow==2.0.0-beta1
# doesn't exist yet
# !pip install -q tensorflow-gpu==2.0
# you'll get a tensorflow-serving-api error
# do your tensorflow-serving (SageMaker work in another session)

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf


In [ ]:
# this won't work with Tensorflow 2.0
# if you have TF 2.0 loaded, you can't set eager - it's forced on

print ('TensorFlow Version:', tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.enable_eager_execution()

## Globals

In [ ]:
PROJECT_DIR = os.getcwd()

IMAGE_DIR = os.path.join(PROJECT_DIR, "data/jpeg_images")
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/annotations")

MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1564778509")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")

# you can get data using the TrainModel_Step1_Local notebook
TEST_TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "code/tfrecords/test/")
                                    
# scratch area
TMP_DIR = os.path.join(PROJECT_DIR, 'tmp')

# NAME - get this from the console
ENDPOINT_NAME = "ep-mobilenet-ssd" 

In [ ]:
# Create a description of the features.
FEATURE_DESCRIPTION = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    # 'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}


## Data

In [ ]:
image_glob = os.path.join(IMAGE_DIR, '*.jpg')
all_image_paths = tf.io.gfile.glob(image_glob)

# inplace shuffle
random.shuffle(all_image_paths)
all_image_paths[0]

## Helper Functions for data type conversions

### Note: 
To stay simple, this example only uses scalar inputs. The simplest way to handle non-scalar features is to use tf.serialize_tensor to convert tensors to binary-strings. Strings are scalars in tensorflow. Use tf.parse_tensor to convert the binary-string back to a tensor.

In [ ]:
# WARNING - this bytes feature seems to be a problem
#         - i am not going to use it - taking the byte feature out of this example
#         - but leaving the function

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
byte_string = b'test_string'
ft_byte_string = _bytes_feature(byte_string)
print("Feature - bytes:", type(ft_byte_string))
print("   value:", ft_byte_string)

# unencoded bytes - encoded to utf-8
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))
print(_int64_feature(True))
print(_int64_feature(1))

### Non-Scalar Example - an image

In [ ]:
img_tensor = tf.io.read_file(all_image_paths[3])
print ("img tensor:", type(img_tensor), img_tensor.dtype)

# serialize the tensor
ser_img_tensor =tf.io.serialize_tensor(img_tensor)
print ("serialized:", type(ser_img_tensor), ser_img_tensor.dtype)

print (img_tensor == ser_img_tensor)

## Create tf.Example message

#### Make up some data

In [ ]:
# the number of observations in the dataset
n_observations = int(1e4)

# boolean feature, encoded as False or True
feature0 = np.random.choice([False, True], n_observations)
print (type(feature0), feature0.shape, feature0[:5])

# integer feature, random from 0 .. 4
feature1 = np.random.randint(0, 5, n_observations)
print (type(feature1), feature1.shape, feature1[:5])

# string feature - note: string as a byte array
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]
print (type(feature2), feature2.shape, feature2[:5])

# float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)
print (type(feature3), feature3.shape, feature3[:5])

### Serialize Example

#### DIFFERENT FROM TUTORIAL
I think this is better :)
- define the feature as a global
- thus, define it in ONE PLACE only
- that way you have consistency everywhere

It nets out the same - but it enforces the idea of having the same format everywhere.


In [ ]:
def make_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """

  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  feature = FEATURE_DESCRIPTION.copy()
  feature['feature0'] = _int64_feature(feature0)
  feature['feature1'] = _int64_feature(feature1)
  # skipping feature 2 - it causes problems
  feature['feature3'] = _float_feature(feature3)
    
#    {
#      'feature0': _int64_feature(feature0),
#      'feature1': _int64_feature(feature1),
#      # 'feature2': _bytes_feature(feature2),   # take bytes feature out
#      'feature3': _float_feature(feature3),
#    }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto

def serialize_example(example_proto):
  return example_proto.SerializeToString()

In [ ]:
# This is an example observation from the dataset.

example_observation = []

example = make_example(False, 4, b'goat', 0.9876)
print ("example protobuf", type(example), '\n', example)

In [ ]:
example_serialized = serialize_example(example)
print ("example serialized", type(example_serialized), '\n', example_serialized)

In [ ]:
predict_dict = {'serialized_example' : example_serialized}
print (type(predict_dict), '\n', predict_dict)

## Decode Example
also called a "message"

In [ ]:
example_deserialized = tf.train.Example.FromString(example_serialized)
print ("deserialized equivalent:", example_deserialized == example)
print (example_deserialized)

## Create Dataset from Slices (numpy arrays)
you can create from a single numpy array  
you can also create with a tuple of arrays

here the Dataset is NOT tf.Example and it's not Tensors

In [ ]:
# feature1 - from above, numpy of int between 1 & 4
print ("feature1", type(feature1), feature1)

# create a Dataset from a numpy array
# from_tensor_slices
ds = tf.data.Dataset.from_tensor_slices(feature1)
print ("created Dataset from numpy array (slice):",type(ds), '\n', ds)

In [ ]:
# create a ds with multiple features
ds = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
print ("created Dataset from numpy array (slice):",type(ds), '\n', ds)

### !! Graph Mode - SKIP THIS if you are in Eager Execution Mode  or TF 2.0 !!
initialize the graph & run it  
-- this won't work with Eager Enabled --

In [ ]:
# This won't work in Eager Execution
# if you really want to run this, set it to
# if True
if False:
    iter = ds.make_initializable_iterator()
    e_tuple = iter.get_next()

    with tf.Session() as sess:
        sess.run(iter.initializer)  # initialize the graph
        result = sess.run(e_tuple)  # run the graph - only once
        print(result)
        f0 = result[0]
        f1 = result[1]
        f2 = result[2]
        f3 = result[3]
        print("feature0:", type(f0), f0)
        print("feature1:", type(f1), f1)

### Eager Mode 
REMEMBER - this notebook will have issues w/ Eager.  But, this will work correctly - take(1) will run just once.   If you run this WITHOUT Eager, it will generate an infinite loop!

In [ ]:
# commented out because you should NOT be in Eager mode
# - but this WILL work fine in Eager mode

# take 1
# as a tuple-4

for t4 in ds.take(1):
    print (type(t4))
    f0,f1,f2,f3 = t4
    print (f0)
    print (f1)
    print (f2)
    print (f3)

### Using Dataset.map - without using Example in this case
you have to take the features and make a tensor.   Pretty clunky

the tutorial says:  
The mapped function must operate in TensorFlow graph mode—it must operate on and return tf.Tensors. A non-tensor function, like create_example, can be wrapped with tf.py_function to make it compatible.

graph mode is the opposite of Eager Execution -- so don't enable Eager Execution.    That also means this stuff is probably going to change with TF 2.0 where Eager is the default. !?!

In [ ]:
# need a function that will take the data in the Dataset and create an example - and serialize it
# I changed the tutorial function so i need to recreate something similiar to make this work

# thus ser_fx_example == tutorial's serialize_example 
def ser_fX_example(feature0,feature1,feature2, feature3):
    """
    Create a tf.Example Message mapping the feature name to a tf.Example-compatible
    """
    #create a dictionary - a copy of the GLOBAL definition
    feature = FEATURE_DESCRIPTION.copy()
    feature['feature0'] = _int64_feature(feature0)
    feature['feature1'] = _int64_feature(feature1)
    # 'feature2' : _bytes_feature(feature2), # take this feature out
    feature['feature3'] = _float_feature(feature3)
    
    # create a f.train.Features()
    features = tf.train.Features(feature=feature)
    # create a tf.train.Example from a Features
    example_proto = tf.train.Example(features=features)
    # serialize the Example - byte array
    example_serialized = example_proto.SerializeToString()
    # this returns a byte string
    return example_serialized

def tf_serialize_example(f0,f1,f2,f3):
    tf_string = tf.py_function(
    ser_fX_example,   # the function
    (f0,f1,f2,f3),    # the function's input
    tf.string)        # the function's output
    # return a Tensor
    return tf.reshape(tf_string,())

#### Test your functions
ser_fX_example: features in -> bytes out  
tf_serialize_example:  takes a function (ser_fX_example), passes the data in and outputs a Tensor of type bytes string

with f2 (the bytes variable) - This will <span style="color:red">NOT WORK</span> if you you have Eager Execution enabled.  You'll get an error where it it has a tensor and it was expecting bytes - or something like that.   

I never fully figured out the root problem.   On StackOverflow, other people had similar problem.   It's not central to what we're trying to do so I took f2 out of this example - and it works fine.

In [ ]:
print ('f0 type:', type(f0), f0.dtype)
print ('f1 type:', type(f1), f1.dtype)
# f2 must be type 
print ('f2 type:', type(f2), f2.dtype)

# -- if Eager, you need this --
f2n = f2.numpy()
print ('f2.numpy() type:', type(f2n))

print ('f3 type:', type(f3), f3.dtype)

s = ser_fX_example(f0,f1,f2,f3)
print ("\nser_fX_example:", type(s), '\n', s)

#### tf.reshape(input, ())
this function will take a byte string and make it a tensor

In [ ]:
byte_string = b'Jay Duff'
print (byte_string)
print (tf.reshape(byte_string, ()))

In [ ]:
# test the function
# - this will work with 1.14 Eager off
# - Won't work with 1.14 Eager on
# - it will probably work with 2.0 Eager On (which is the default)

# commented out since you should have Eager on
# serialized_feature_example = tf_serialize_example(f0,f1,f2,f3)
# print (type(serialized_feature_example))
# print (serialized_feature_example)

In [ ]:
serialized_features_dataset = ds.map(tf_serialize_example)
print (serialized_features_dataset)

## Create the Dataset from a Generator
This is an alternative to creating a Dataset from slices

### Generator
Using a generator means the sequence is NOT stored in memory

In [ ]:
def generator():
    for features in ds:
        yield ser_fX_example(*features)

In [ ]:
serialized_features_dataset2 = tf.data.Dataset.from_generator(generator, output_types=tf.string, output_shapes=())

In [ ]:
print (serialized_features_dataset2)
print (serialized_features_dataset == serialized_features_dataset)

## Working with TF Records - Part 1 - tf.data.experimental 

#### <span style="color:red">CONCEPTS ONLY - tf.data.experimental is NOT the way to do this in production.   See TFRecords Part 2 with tf.io</span>  
Not sure why the tutorial goes through tf.data.experimental ???  
Below we repeat the same exercises with tf.io  
### Write TF Record File

In [ ]:
# you can write a Dataset of serialized examples to a TFRecord file
tfrecord_file_output = os.path.join(TMP_DIR, 'tutorial.tfrecord')
print (tfrecord_file_output)

# create the writer 
writer = tf.data.experimental.TFRecordWriter(tfrecord_file_output)
writer.write(serialized_features_dataset)


### Read TF Record File
We have 2 files - we just put this tutorial TFRecord file in /tmp -- because we don't care about it.  
The ssd-project (w/ CFA Product data) has train/val/test tfrecord files in /data/tfrecord directory

In [ ]:
# Tutorial tfrecord file (from above)
tfrecord_file_exists = tf.io.gfile.exists(tfrecord_file_output)
print ("tfrecords file path - exists:", tfrecord_file_exists, tfrecord_file_output)

In [ ]:
# CFA Product Images - for SSD
# this will be a different tf.Example structure than this tutorial
tfrecord_file_exists = tf.io.gfile.exists(TEST_TFRECORDS_PATH)
print ("tfrecords file path - exists:", tfrecord_file_exists, TEST_TFRECORDS_PATH)

### Tutorial Dataset / tfrecord
This notebook will continue with the simple tf.Example where Feature = {feature0, feature1, feature3}.  Remember, we dropped feature2.

Feature is based on whatever format YOU elect.   It's NOT a self-describing format (not good!)  So it's important YOU keep track of the format throughout these pipelines.   in parallel, there is a notebook:  
#### UnderstandingObjectDetectionExample
This notebook will have equivalent operations with the SSD tf.Example (which stores an image).  The tf.Example in this notebook is based on the original MobileNet model - and of course it's different than this tutorial

In [ ]:
# get length of dataset
# - at this time, there is no simple function
#   this is brute force - ii just want this for QA reasons

def get_dataset_length(ds):
    num_elements = 0
    for element in ds:
        num_elements += 1
    return num_elements

In [ ]:
# tutorial tfrecord file
# - note - this is a list of files, you should shard your files if there are more than 5,000
tfrecord_filenames = [tfrecord_file_output]
raw_dataset = tf.data.TFRecordDataset(tfrecord_filenames)
print ("tfrecord file:", type(raw_dataset), '\n', raw_dataset)

# cache the dataset in memory
raw_dataset.cache()
print ("dataset size:", get_dataset_length(raw_dataset))

### Parsing Serialized Data

#### TF 1.14 vs 2.0
- 1.14:  single record is DatasetV1Adapter
- 2.0 :  it is a TakeDataset

You can't take a single record in either version and do:
single_record.numpy()  

But you can iterate

In [ ]:
single_record = raw_dataset.take(1)
print ("record from file:", type(single_record), '\n', single_record)

### iterating through the dataset

#### TF 1.14 & TF 2.0
EagerTensor - and you can get the contents w/ .numpy()

In [ ]:
for raw_record in raw_dataset.take(3):
    print (type(raw_record))
    print (repr(raw_record))
    print ("contents:", raw_record.numpy())
    # the contents are a serialized example?
    print ('\n')

### Parsing
you have EagerTensors, you have to parse the serialized data (deserialize) back to a feature.  So its important you use the SAME feature definition as when you created the dataset (and wrote it)


In [ ]:
print (type(FEATURE_DESCRIPTION), 'n', FEATURE_DESCRIPTION)

In [ ]:
def _parse_function(example_proto):
    # Parse the input using the standard dictionary
    feature = tf.io.parse_single_example(example_proto, FEATURE_DESCRIPTION)
    return feature

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
print (parsed_dataset)

In [ ]:
# now you see your dataset is features
for parsed_record in parsed_dataset.take(3):
    print(repr(parsed_record))

## TF Records - Part 2 - tf.io
This should be the preferred way.   It's safe to say tf.data.experimental is not long term!!  Not sure why the tutorial included experimental.  (gee, that was a waste of time)

### Recreate a TFRecord file

In [ ]:
# new tfrecord file
tfrecord_file_output = os.path.join(TMP_DIR, 'tutorial_part2.tfrecord')
print (tfrecord_file_output)

with tf.io.TFRecordWriter(tfrecord_file_output) as writer:
        for i in range(n_observations):
            example = ser_fX_example(feature0[i], feature1[i], feature2[i], feature3[i])
            writer.write(example)
            
! ls -l {tfrecord_file_output}
! du -sh {tfrecord_file_output}

### Read the TFRecord File


In [ ]:
tfrecord_file_list_input = [tfrecord_file_output]
raw_dataset = tf.data.TFRecordDataset(tfrecord_file_list_input)
print (type(raw_dataset))

In [ ]:
for raw_record in raw_dataset.take(1):
    print("raw record type:", type(raw_record))  # serialized Example
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())  # Parse will de-serialize it
    print(example)

## TFServing (and SageMaker Endpoint)
SageMaker endpoint is based on TensorFlow Serving.  

Assuming you have an active endpoint - in this case a mobilenet-ssd model - CLEARLY NOT the model for this bogus data structure.  The desire here is simply to test the data format & serialization.


In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.predictor import json_serializer, json_deserializer

sagemaker_session = sagemaker.Session()
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(ENDPOINT_NAME, sagemaker_session)

print (type(predictor))

In [ ]:
import json


for raw_record in raw_dataset.take(3):
    print("raw record type:", type(raw_record))  # serialized Example
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print (example)
    #
    predict_request = {'serialized_example' : repr(example)}
    predict_string = json.dumps(predict_request)
    predict_json = json.loads(predict_string)
    output_dict = predictor.predict(predict_string)